In [1]:
from __future__ import print_function

import random

import pandas as pd
from tqdm import tqdm
import numpy as np

In [3]:
from utils.common import combine_csvs

combine_csvs('output/ards_data_dynamic', 'valid_ards_data_with_dynamic_feature_9399.csv')
print('end')

There are 2 files
     icu_stay_id identification_offset  AST (SGOT)_median  \
0         392433                  4687                NaN   
1         216421                  1641              194.0   
2         364289                  1357                NaN   
3        3335357                   459                NaN   
4         948534                  1155               31.0   
...          ...                   ...                ...   
4995     1506026                  1099               59.0   
4996     1704142                   766                NaN   
4997     2851257                   228              108.0   
4998     2716534                  3488              239.0   
4999      601385                   334                NaN   

      AST (SGOT)_variance  AST (SGOT)_rate_change  albumin_median  \
0                     NaN                     NaN             NaN   
1                     NaN                     0.0             2.1   
2                     NaN                 

In [51]:
df = pd.read_csv('dataset/figure_data/fig4_data.csv')
df['P/F ratio_rate_change']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
        ...  
9394    27.40
9395     8.29
9396      NaN
9397      NaN
9398      NaN
Name: P/F ratio_rate_change, Length: 9399, dtype: float64

In [35]:
# a = df.loc[:, 'pf_8h_min']
df['pf_8h_min'] = df['pf_8h_min'].map(lambda x: np.floor(x))
df['pf_8h_min'].value_counts()

1.0     1812
2.0     1469
0.0      880
3.0      767
4.0      305
5.0       64
6.0        6
12.0       2
7.0        2
13.0       1
8.0        1
15.0       1
16.0       1
Name: pf_8h_min, dtype: int64

In [17]:
from utils.postgres_sql import PostgresSqlConnector

test_icu_stay_id = 1502830
start_offset = 666
end_offset = start_offset + 24 * 60
sql_connector = PostgresSqlConnector()

In [19]:
sql_connector.get_pao2_fio2_in_first_8h_after_ards_identification(
test_icu_stay_id, start_offset
)

,time_offset,label,value
0,666,paO2,30.0
1,666,FiO2,100.0
